In [108]:
#%ngrok authtoken 28O6b8H568ljQHMB6iNMekjSBh3_4oC5ziznYkA2AxbW6GrHG

'''
installs Flask, OpenCV, Pickle, and Tensorflow if needed
'''
%pip install Flask-Session
%pip install Flask
%pip install opencv-python
%pip install pickle-mixin
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [111]:
from keras.utils.np_utils import to_categorical
import json
import base64
from flask import Flask, url_for,request,render_template
from flask_ngrok import run_with_ngrok
import os
import pickle
import sys
import io

from tensorflow import keras
from keras import optimizers
from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras.backend import dtype
from flask import jsonify
from PIL import Image, ImageOps
import numpy as np
import requests
import cv2
#from google.colab.patches import cv2_imshow

#from google.colab import drive
#drive.mount('/content/gdrive')
#prefix = '/content/drive/My Drive/'

#custom_path = 'Colab Notebooks/ML Final Project'
#sys_path = prefix + custom_path
#sys.path.append(sys_path)#

#htmlfile = os.path.join(sys_path, 'finalproj.html')

In [112]:
batch_size = 32
num_classes = 10
epochs = 25

In [113]:
'''
function to load data from MNIST dataset and split into training, testing, and validation data sets
'''
def generate_data():
  (x_train, y_train), (x_test, y_test) = mnist.load_data()
  
  y_train = to_categorical(y_train,num_classes)
  y_test = to_categorical(y_test,num_classes)

  x_train = x_train.astype('float32')
  x_test = x_test.astype('float32')
  x_train /= 255
  x_test /= 255

  #splits the data for training and validation
  x_validate = x_train[50000:,:]
  x_train = x_train[:50000,:]
  y_validate = y_train[50000:,:]
  y_train = y_train[:50000,:]

  #reshapes image data sets
  x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
  x_test = x_test.reshape((x_test.shape[0], 28, 28, 1))
  
  return x_train, y_train, x_validate, y_validate, x_test, y_test

In [114]:
x_train, y_train, x_validate, y_validate, x_test, y_test = generate_data()

In [115]:
def cnn():

  model = Sequential()

  #stack of convolution layers and sets input shape
  model.add(Conv2D(32, (3, 3), padding='same',input_shape=(28, 28, 1)))
  model.add(Activation('relu'))
  model.add(Conv2D(32, (3, 3), padding='same'))
  model.add(Activation('relu'))

  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  #second stack of convolutional layers
  model.add(Conv2D(64, (3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(64, (3, 3), padding='same'))
  model.add(Activation('relu'))

  model.add(MaxPooling2D(pool_size=(2, 2)))


  model.add(Flatten())
  model.add(Dense(256))
  model.add(Activation('relu'))
  model.add(Dropout(0.25))
  model.add(Dense(num_classes))
  model.add(Activation('softmax'))

  opt = keras.optimizers.Adam(learning_rate=0.001)
  
  model.compile(loss='categorical_crossentropy',
                optimizer=opt,
                metrics=['accuracy'])
  print(model.summary())

  return model

In [101]:
#runs CNN model and evaluates it
model = cnn()
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_validate, y_validate))
_, acc = model.evaluate(x_test,y_test,verbose=0)
print()
print((acc * 100.0))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 28, 28, 32)        320       
                                                                 
 activation_12 (Activation)  (None, 28, 28, 32)        0         
                                                                 
 conv2d_9 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 activation_13 (Activation)  (None, 28, 28, 32)        0         
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 14, 14, 32)        0         
                                                      

In [116]:
'''
K-Nearest Neighbors Algorithm for images
Takes an image of size (1,28.28,1) and returns its prediction
'''
def knn_predict(img):
  k = 10
  preds = []
  count = {}
  
  distances = []
  #finds distance between input image and each image in the train dataset
  for j in range(len(x_train)):
    dist = np.sqrt(np.sum((x_train[j]-img)**2))
    idx = np.where(y_train[j] == 1)
    #stores distance with training image label
    distances.append((idx[0][0],dist))

  #sorts the images from shortest distance to longest 
  distances.sort(key=lambda x:x[1])
  labels = []
  
  #gets the k neighborhood
  for val in range(k):
    labels.append(distances[val][0])

  count = {}
  for label in labels:
    if label not in count:
      count[label] = 1
    else:
      count[label] += 1
  maximum = max(count.values())
   
  for item in count.items():
    if item[1] == maximum:
      return item[0]

In [117]:
'''
runs KNN predictions over the first 2000 images of the test dataset
'''
correct = 0
acc_list = []
for i in range(len(x_test)):
  #print(i)
  if i > 2000:
    break
  pred = knn_predict(x_test[i])
  #print(pred)
  #print(y_test[i])
  temp = np.where(y_test[i] == 1)
  if pred == temp:
    correct += 1
  acc = (correct / (i+1)) * 100
  acc_list.append(acc)

In [118]:
avg_acc = sum(acc_list)/len(acc_list)
final_acc = acc_list[-1]

print(avg_acc)
print(final_acc)

96.33758160178428
95.40229885057471


In [119]:
'''
saves the CNN Model as .pkl file to be used
'''
with open('CNN.pkl','wb') as file:
  pickle.dump(model,file)

INFO:tensorflow:Assets written to: ram://01392851-5e57-4a48-814c-f67ca64e2b92/assets


In [120]:
'''
This block contains creates the Flask application webpage
and receives Post requests when a prediction is ready to be made.
Once the prediction has been made, both results are return in a json format.
'''
#edit this path variable to where the .ipynb, .pkl, and .html files are stored on your system
path = '/Users/mathu/Documents/Drawing'
app = Flask(__name__, template_folder=path)
#run_with_ngrok(app)   

#with open('CNN.pkl','rb') as f:
#  model = pickle.load(f)

#main page of flask using created html as a template
@app.route("/")
def home():
    return render_template('finalproj.html')

#this receives post requests from the html and performs
#all necessary operations to evaluate the image and then
#returns it serialized as a json
@app.route('/postmethod',methods=['POST'])
def getImageData():
  #stores post request
  jsonDataBytes= request.data
  jsonData = jsonDataBytes.decode('utf-8')
  j = json.loads(jsonData)
  img = j['img']
  imgsplit = img.split(',')[1]
  temp = base64.b64decode(imgsplit)
  #gets decoded image and begins converting it to be used by the models
  drawn = Image.open(io.BytesIO(temp))
  
  image = np.asarray(drawn)
  g = ImageOps.grayscale(drawn)
  arr = np.asarray(g)
  cv2.imshow('',arr)
  newimage = cv2.resize(arr,(28,28),cv2.INTER_AREA)
  imagetopredict = np.asarray(newimage, dtype="uint8")
  imagetopredict = imagetopredict.reshape(1, 28, 28, 1).astype('float32')
  
  pred = model.predict(imagetopredict)[0]

  pred1 = knn_predict(imagetopredict)
  return jsonify(result=str(pred1),result1=str(np.argmax(pred)))
app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [04/May/2022 16:39:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2022 16:39:10] "POST /postmethod HTTP/1.1" 200 -
